In [10]:
from tkinter import Tk, Canvas, Button, PhotoImage, Radiobutton, StringVar, Scale
from random import sample

cellsize = 25
lst_mines = []

def random_mines(p,g):
    size = len(g)
    nmines = int(size*p)
    global compteurMines
    compteurMines = nmines
    mines = sample(g, nmines)
    return mines

def voisins(n, i, j):
    "Dans une grille n x n, indices (ligne, colonne) des voisins dans la grille d'un sommet (i,j) de la grille"
    return [(a,b) for (a, b) in [(i, j+1),(i, j-1), (i-1, j), (i+1,j), (i+1,j-1),(i+1,j+1),(i-1,j+1),(i-1,j-1)] if a in range(n) and b in range(n)]

def genere(n,cx,cy,i):
    L = []
    for x in range(0,2*i+1):
        for y in range(0,2*i+1):
            if x==0 or x==2*i or y==0 or y==2*i:
                L.append((x+cx-i,y+cy-i))
    return L

def nombre(mines,n,click):
    grid = [[0]*n for _ in range(n)]
    for (i,j) in mines:
        v = voisins(n,i,j)
        for (a,b) in v:
            if not((a,b) in mines):
                grid[a][b] += 1
                
    for (x,y) in mines:
        grid[x][y] = "X"
    
    return grid

def drawMenu(cnv):
    global widgets
    niveau = StringVar()
    niveau.set('normal')
    widgets.append(niveau)

    cnv.create_image(MAX_X/2,MAX_Y/2,image=imgMenu)    
    facile = Radiobutton(root, variable=niveau, text="Facile",value="facile", font="arial 10")
    facile.pack(side="left", expand=1)
    widgets.append(facile)

    normal = Radiobutton(root, variable=niveau, text="Normal",value="normal", font="arial 10")
    normal.pack(side="left", expand=1)
    widgets.append(normal)

    difficile = Radiobutton(root, variable=niveau, text="Difficile",value="difficile", font="arial 10")
    difficile.pack(side="left", expand=1)
    widgets.append(difficile)

    #slider = Scale(root, orient = "horizontal", command=taillePlateau, from_=20, to=100)
    #slider.pack()
    #widgets.append(slider)

def start(event):
    if screen == "game":
        click = (event.y//cellsize,event.x//cellsize)
        if lst_mines==[]:
            global states
            states = createMap(click,p)
            reGenMap()
            timer()
        if bPerdu == False:
            revelle(n,states,click)
            
def startMenu():
    niv = widgets[0].get()
    global p, screen
    if niv == "facile":
        p = 0.05
    elif niv == "normal":
        p = 0.15
    else:
        p = 0.2
    screen = "game"
    grille(n)
    for i in range(1,len(widgets)):
        widgets[i].destroy()
    
def createMap(click,p):
    grid1,p = update(n,p,click)

    #On génère une grille normalement avec toutes les mines de taille n
    grid = [(line,col) for col in range(0,n) for line in range(0,n)]
    mines = random_mines(p,grid)
    grid2 =[[0]*n for _ in range(n)]
    for (a,b) in mines:
        grid2[a][b] = "X"
    #On supperpose les deux grilles states, en faisant attention que grid1 soit prioritaire sur grid2
    map=[]
    for x in range(n):
        map.append([])
        for y in range(n):
            if (x>=click[0]-4 and x<=click[0]+4) and (y>=click[1]-4 and y<=click[1]+4):
                map[x].append(grid1[x][y])
            else:
                map[x].append(grid2[x][y])
                if grid2[x][y]=="X":
                    lst_mines.append((x,y))
    N = nombre(lst_mines,n,click)
    afficher(N)
    return N

def afficher(grid):
    for i in range(len(grid)):
        for j in range(len(grid[0])):
            print(grid[i][j], end = " ")
        print()

#update permet de créer un grille de 4x4 au début pour créer une zones sans mines en fonction du click du joueur
def update(n,p,click):
    for i in range(1,4):
        if click[0]+i or click[0]-i or click[1]+i or click[1]-i:
            #genere permet de créer une liste qui contient 
            #les coordonnées des cases autour du click en fonction de i
            g = genere(n, click[0], click[1],i)
            mines = random_mines(p, g)
            p += 0.05
            for (a,b) in mines:
                lst_mines.append((a,b))
    
    grid=[[0]*n for _ in range(n)]
    for (a,b) in lst_mines:
        grid[a][b] = "X"
    return grid,p

def grille(n):
    cnv.delete("all")
    map = []
    for i in range(n):
        map.append([])
        for j in range(n):
            map[i].append(9)
    draw(map)

def draw(map):
    for i in range(len(map)):
        for j in range(len(map[0])):
            if map[i][j]=="X":
                cnv.create_image(cellsize*(i+0.5),cellsize*(j+0.5),image=imgMine)
            elif map[i][j] == 9 and bPerdu == False:
                cnv.create_rectangle((j*cellsize,i*cellsize), ((j+1)*cellsize,(i+1)*cellsize), fill="gray", outline='black')
    
    if bPerdu == True:
        cnv.create_text(MAX_X/2, MAX_Y/2, text="PERDU!", font=('courier', 80, 'bold'), fill="red")
    elif bGagné == True:
        cnv.create_text(MAX_X/2, MAX_Y/2, text="GAGNE!", font=('courier', 80, 'bold'), fill="red")

def decouvreVoisins(p,grid,L):
    v = voisins(n,p[0],p[1])
    v.append((p[0],p[1]))
    for (i,j) in v:
        if mapFlag[i][j] != 1 and mapFlag[i][j] != 2 and grid[i][j] != "X":
            lst_caseDecouvertes.append((i,j))
            caseDecouverte[i][j] = 1
            if grid[i][j] == 0:
                    L.append((i,j))
    return L

def revelle(n,grid,click):
    nombreDeTours = int((0.2//p)+1)
    x,y = click[0],click[1]
    compteurDeDrapeau = 0
    mineVoisin = 0
    L = []
    global lst_caseDecouvertes,caseDecouverte
    if caseDecouverte[x][y] == 1 or lst_caseDecouvertes == []:
        v = voisins(n,y,x)
        for i,j in v:
            if mapFlag[i][j] == 1:
                compteurDeDrapeau += 1
            elif grid[j][i] == "X" and mapFlag[j][i] != 2 and lst_caseDecouvertes != []:
                mineVoisin = (j,i)
        if compteurDeDrapeau == grid[x][y] or lst_caseDecouvertes == []:
            if mineVoisin != 0:
                perdu(mineVoisin)
            L = decouvreVoisins(click,grid,L)         
            for k in range(nombreDeTours):
                for (i,j) in L:
                    L = decouvreVoisins((i,j),grid,L)
                    grid[i][j] = 9
                    L.remove((i,j))
    else:
        if grid[x][y] == "X":
            perdu(click)
        else:
            lst_caseDecouvertes.append((x,y))
            caseDecouverte[x][y] = 1
            
def perdu(click):
    if states[click[0]][click[1]] == "X":
        bPerdu = True
        draw(states)
        highScores(chrono)
        
def gagné(flag, states):
    for (i,j) in flag:
        if states[i][j] != "X":
            return False
    return True

def reGenMap():
    if bPerdu == False:
        cnv.delete("all")
        grille(n)
        
        for i in range(len(mapFlag)):
            for j in range(len(mapFlag)):
                if mapFlag[i][j] == 1:
                    cnv.create_image(cellsize*(i+0.5),cellsize*(j+0.5),image=imgDrapeau)
                elif mapFlag[i][j] == 2:
                    cnv.create_image(cellsize*(i+0.5),cellsize*(j+0.5),image=imgInterrogation)
        
        for (i,j) in lst_caseDecouvertes:
            if not((i,j) in lst_mines):
                cnv.create_rectangle((j*cellsize,i*cellsize), ((j+1)*cellsize,(i+1)*cellsize), fill="white", outline='black')
                if states[i][j] != 9:
                    cnv.create_text(j*R+R//2, i*R+R//2, text=states[i][j], font=('courier', MAX_X//35, 'bold'))

        #Afficher le chronomètre dans un afficheur SSD         
        afficheur(44,64,8,splitNumber(chrono)[2])
        afficheur(50,64,8,splitNumber(chrono)[1])
        afficheur(56,64,8,splitNumber(chrono)[0])

        #Afficher le compteur de mine dans un afficheur SSD
        afficheur(1,64,8,splitNumber(compteurMines-len(flag))[2])
        afficheur(7,64,8,splitNumber(compteurMines-len(flag))[1])
        afficheur(13,64,8,splitNumber(compteurMines-len(flag))[0])

        cnv.after(50,reGenMap)

        
def drapeau(event):
    if screen == "game":
        x, y = event.x//cellsize, event.y//cellsize
        global mapFlag
        if mapFlag[x][y] == 1:
            mapFlag[x][y] = 2
            flag.remove((x,y))
        elif mapFlag[x][y] == 2:
            mapFlag[x][y] = 0
        elif caseDecouverte[y][x] != 1:
            flag.append((x,y))
            mapFlag[x][y] = 1
        
        if compteurMines-len(flag) == 0:
            bGagne = gagné(flag, states)

    
def timer():
    global chrono
    chrono += 1
    cnv.after(1000,timer)
    
def afficheur(x,y,t,nb):
    if nb == 0 or nb == 2 or nb == 3 or nb == 5 or nb == 7 or nb == 8 or nb == 9:
        cnv.create_polygon((x*t,y*t),((x+5)*t,y*t),((x+4)*t,(y+1)*t),((x+1)*t,(y+1)*t),fill="red",outline='black')
    
    if nb == 0 or nb == 4 or nb == 5 or nb == 6 or nb == 8 or nb == 9:
        cnv.create_polygon((x*t,y*t),(x*t,(y+5)*t),((x+1)*t,(y+4.5)*t),((x+1)*t,(y+1)*t),fill="red",outline='black')
    
    if nb == 0 or nb == 1 or nb == 2 or nb == 3 or nb == 4 or nb == 7 or nb == 8 or nb == 9:
        cnv.create_polygon(((x+5)*t,y*t),((x+5)*t,(y+5)*t),((x+4)*t,(y+4.5)*t),((x+4)*t,(y+1)*t),fill="red",outline='black')
    
    if nb == 2 or nb == 3 or nb == 4 or nb == 5 or nb == 6 or nb == 8 or nb == 9:
        cnv.create_polygon((x*t,(y+5)*t),((x+1)*t,(y+5.5)*t),((x+4)*t,(y+5.5)*t),((x+5)*t,(y+5)*t),((x+4)*t,(y+4.5)*t),((x+1)*t,(y+4.5)*t),fill="red",outline='black')
    
    if nb == 0 or nb == 2 or nb == 6 or nb == 8:
        cnv.create_polygon((x*t,(y+5)*t),(x*t,(y+10)*t),((x+1)*t,(y+9)*t),((x+1)*t,(y+5.5)*t),fill="red",outline='black')
    
    if nb == 0 or nb == 1 or nb == 3 or nb == 4 or nb == 5 or nb == 6 or nb == 7 or nb == 8 or nb == 9:
        cnv.create_polygon(((x+5)*t,(y+5)*t),((x+5)*t,(y+10)*t),((x+4)*t,(y+9)*t),((x+4)*t,(y+5.5)*t),fill="red",outline='black')
    
    if nb == 0 or nb == 2 or nb == 3 or nb == 5 or nb == 6 or nb == 8:
        cnv.create_polygon((x*t,(y+10)*t),((x+5)*t,(y+10)*t),((x+4)*t,(y+9)*t),((x+1)*t,(y+9)*t),fill="red",outline='black')

def splitNumber(nb):
    L = []
    L.append(nb%10)
    nb = nb//10
    L.append(nb%10)
    nb = nb//10
    L.append(nb)
    return L

def reset():
    lst_caseDecouvertes = []
    chrono = 0
    reGenMap()
    
def highScores():
    #Ecrire dans le HighScore tous les scores (append) et les parcourirs pour déterminer le meilleurs temps
    fichier = open("./HighScores.txt", 'r')
    score=0
    if fichier.readline() != "":
        score = int(fichier.readline())
    fichier.close()
    fichier = open("./HighScores.txt", "w")
    if score > chrono or score == 0:
        fichier.write(str(chrono))
    fichier.close()
    
def createCanvas(t):
    MAX_X,MAX_Y = n*cellsize,n*cellsize+100
    cnv=Canvas(root, width=MAX_X, height=MAX_Y, bg="gray")
    cnv.pack()
    return cnv,MAX_X,MAX_Y
    
global n
global cnv
n = 20
MAX_X,MAX_Y = n*cellsize,n*cellsize+100
R = MAX_X//n
states = []
mapFlag = [[0]*n for _ in range(n)]
lst_caseDecouvertes = []
global bGagné, bPerdu
bPerdu = False
bGagné = False
chrono = 0
flag = []
widgets = []
caseDecouverte = [[0]*n for i in range(n)]
global screen
screen = "menu"

root=Tk()
global cnv
cnv,MAX_X,MAX_Y = createCanvas(n)

imgDrapeau = PhotoImage(file="drapeau.gif")
imgMine = PhotoImage(file="mine.gif")
imgInterrogation = PhotoImage(file="interrogation.gif")
imgMenu = PhotoImage(file="démineur.gif")

if screen == "game":
    btn=Button(root, text="Reset", command=reset)
    btn.pack()
else:
    drawMenu(cnv)
    btnStart=Button(root, text="Start", command=startMenu)
    btnStart.pack()
    widgets.append(btnStart)

root.bind("<Button-1>", start)
root.bind("<Button-3>", drapeau)


root.mainloop()



0 1 X 3 X X X X 3 X X 1 1 2 4 X X X X 1 
0 1 1 3 X 4 3 2 3 X 4 3 3 X X X 6 X 3 1 
0 0 0 2 2 2 0 0 1 2 X 3 X X 6 X 3 2 2 1 
1 1 0 1 X 2 1 0 1 3 3 4 X 5 X 3 2 2 X 2 
X 2 0 1 2 X 1 1 2 X X 3 3 X 3 X 1 3 X 3 
X 3 0 1 2 2 1 1 X 3 3 X 2 1 2 1 1 2 X 2 
X 3 1 2 X 3 1 2 1 1 1 1 1 0 0 1 1 2 1 1 
3 X 1 2 X 3 X 1 0 0 0 1 1 2 1 2 X 3 2 1 
X 2 2 2 2 3 3 3 1 0 1 2 X 2 X 3 4 X X 1 
3 4 4 X 3 3 X X 1 0 1 X 2 3 2 3 X X 3 1 
X X X X X 3 X 3 1 1 3 3 2 1 X 2 2 2 1 0 
X 7 5 4 2 2 1 1 0 1 X X 1 1 1 2 2 2 1 0 
X X X 3 1 0 0 0 0 2 4 4 2 0 0 1 X X 2 1 
4 6 X X 2 1 0 0 0 1 X X 1 1 2 4 5 5 4 X 
X X 3 3 X 2 1 1 1 2 3 3 2 2 X X X X X 2 
X 4 1 1 1 3 X 2 2 X 2 1 X 2 2 4 5 5 3 1 
X 3 1 1 1 2 X 2 3 X 4 3 3 2 0 1 X X 1 0 
X 3 2 X 1 1 1 1 3 X 5 X X 1 1 2 3 2 1 0 
1 2 X 3 2 0 0 1 3 X X 4 2 1 2 X 3 2 2 1 
0 1 2 X 1 0 0 1 X 4 X 2 0 0 2 X 3 X X 1 
